In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

from wordcloud import WordCloud
#if encounter error: No module named 'wordcloud'
#run the line below

import folium
from folium.plugins import HeatMap
#if encounter error: No module named 'folium'
#run the line below

import plotly.offline as pyo
import plotly.graph_objs as go
#if encounter error: No module named 'plotly'
#run the line below

import seaborn as sns
sns.set_style('whitegrid')

In [2]:
# Import the dataset
airbnb_data = pd.read_csv("PredictPrice.csv")
airbnb_df = pd.DataFrame(airbnb_data)
airbnb_data.head(5)

,id,new_host_acceptance_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,bedrooms,beds,number_of_reviews,availability_365,price
0,50646.0,65.0,f,Bukit Timah,Central Region,1.33432,103.78521,Private room,1.0,1.0,18.0,365.0,80.0
1,71609.0,65.0,f,Tampines,East Region,1.34537,103.95887,Private room,2.0,3.0,20.0,365.0,179.0
2,71896.0,65.0,f,Tampines,East Region,1.34754,103.95958,Private room,1.0,1.0,24.0,365.0,82.0
3,71903.0,65.0,f,Tampines,East Region,1.34531,103.96100,Private room,1.0,2.0,47.0,365.0,82.0
4,275343.0,77.0,f,Bukit Merah,Central Region,1.29015,103.80814,Private room,1.0,1.0,20.0,341.0,52.0


In [3]:
airbnb_data[['price','new_host_acceptance_rate', 'bedrooms', 'beds', 'number_of_reviews', 'availability_365']].describe()

,price,new_host_acceptance_rate,bedrooms,beds,number_of_reviews,availability_365
count,3671.000000,3671.000000,3671.000000,3671.000000,3671.000000,3671.000000
mean,206.755652,56.200490,1.182784,1.791882,11.777172,225.244348
std,370.691181,36.966118,0.767236,2.168451,31.865084,145.968915
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,70.000000,30.000000,1.000000,1.000000,0.000000,81.000000
50%,135.000000,65.000000,1.000000,1.000000,1.000000,311.000000
75%,220.000000,91.000000,1.000000,2.000000,7.000000,362.000000
max,8900.000000,100.000000,8.000000,46.000000,369.000000,365.000000


__we can see a huge difference in the range of values present in our numerical features:
price,beds,number of reviews and availability_365. We will use scaling to fix this problem__
__to be done during the model building phase__

things to do be done before model building
1) Normalization is a scaling technique in which values are shifted and rescaled so that they end up ranging between 0 and 1. It is also known as Min-Max scaling. Feature scaling to be done later when tuning model for comparsion
2) Convert categorical variable into numbers some  ML libraries do not take categorical variables as input. Label encoder (neigbourhood and region)

In [4]:
airbnb_df

,id,new_host_acceptance_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,bedrooms,beds,number_of_reviews,availability_365,price
0,50646.0,65.0,f,Bukit Timah,Central Region,1.334320,103.785210,Private room,1.0,1.0,18.0,365.0,80.0
1,71609.0,65.0,f,Tampines,East Region,1.345370,103.958870,Private room,2.0,3.0,20.0,365.0,179.0
2,71896.0,65.0,f,Tampines,East Region,1.347540,103.959580,Private room,1.0,1.0,24.0,365.0,82.0
3,71903.0,65.0,f,Tampines,East Region,1.345310,103.961000,Private room,1.0,2.0,47.0,365.0,82.0
4,275343.0,77.0,f,Bukit Merah,Central Region,1.290150,103.808140,Private room,1.0,1.0,20.0,341.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,53905492.0,100.0,f,Bukit Merah,Central Region,1.286680,103.834440,Private room,1.0,2.0,0.0,358.0,185.0
3667,53905653.0,100.0,f,Bukit Merah,Central Region,1.285092,103.833326,Private room,1.0,2.0,0.0,361.0,215.0
3668,53910005.0,0.0,f,Marine Parade,Central Region,1.302760,103.901580,Private room,1.0,1.0,0.0,365.0,48.0
3669,53910434.0,30.0,f,Kallang,Central Region,1.300063,103.859894,Entire home/apt,2.0,3.0,0.0,363.0,529.0


In [5]:
#remove rows that are zero
airbnb_df = airbnb_df.loc[(airbnb_df['price'] > 0)]

In [6]:
from sklearn import preprocessing
category_col =['neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'room_type','host_is_superhost']
labelEncoder = preprocessing.LabelEncoder()
mapping_dict ={}
for col in category_col:
    airbnb_df[col] = labelEncoder.fit_transform(airbnb_df[col])
    le_name_mapping = dict(zip(labelEncoder.classes_,
                        labelEncoder.transform(labelEncoder.classes_)))
    mapping_dict[col]= le_name_mapping
print(mapping_dict)


{'neighbourhood_cleansed': {'Ang Mo Kio': 0, 'Bedok': 1, 'Bishan': 2, 'Bukit Batok': 3, 'Bukit Merah': 4, 'Bukit Panjang': 5, 'Bukit Timah': 6, 'Central Water Catchment': 7, 'Choa Chu Kang': 8, 'Clementi': 9, 'Downtown Core': 10, 'Geylang': 11, 'Hougang': 12, 'Jurong East': 13, 'Jurong West': 14, 'Kallang': 15, 'Lim Chu Kang': 16, 'Mandai': 17, 'Marina South': 18, 'Marine Parade': 19, 'Museum': 20, 'Newton': 21, 'Novena': 22, 'Orchard': 23, 'Outram': 24, 'Pasir Ris': 25, 'Pioneer': 26, 'Punggol': 27, 'Queenstown': 28, 'River Valley': 29, 'Rochor': 30, 'Sembawang': 31, 'Sengkang': 32, 'Serangoon': 33, 'Singapore River': 34, 'Southern Islands': 35, 'Sungei Kadut': 36, 'Tampines': 37, 'Tanglin': 38, 'Toa Payoh': 39, 'Western Water Catchment': 40, 'Woodlands': 41, 'Yishun': 42}, 'neighbourhood_group_cleansed': {'Central Region': 0, 'East Region': 1, 'North Region': 2, 'North-East Region': 3, 'West Region': 4}, 'room_type': {'Entire home/apt': 0, 'Hotel room': 1, 'Private room': 2, 'Shared 

C:\Users\liang\AppData\Local\Temp/ipykernel_7448/206942947.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
airbnb_df

,id,new_host_acceptance_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,bedrooms,beds,number_of_reviews,availability_365,price
0,50646.0,65.0,0,6,0,1.334320,103.785210,2,1.0,1.0,18.0,365.0,80.0
1,71609.0,65.0,0,37,1,1.345370,103.958870,2,2.0,3.0,20.0,365.0,179.0
2,71896.0,65.0,0,37,1,1.347540,103.959580,2,1.0,1.0,24.0,365.0,82.0
3,71903.0,65.0,0,37,1,1.345310,103.961000,2,1.0,2.0,47.0,365.0,82.0
4,275343.0,77.0,0,4,0,1.290150,103.808140,2,1.0,1.0,20.0,341.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,53905492.0,100.0,0,4,0,1.286680,103.834440,2,1.0,2.0,0.0,358.0,185.0
3667,53905653.0,100.0,0,4,0,1.285092,103.833326,2,1.0,2.0,0.0,361.0,215.0
3668,53910005.0,0.0,0,19,0,1.302760,103.901580,2,1.0,1.0,0.0,365.0,48.0
3669,53910434.0,30.0,0,15,0,1.300063,103.859894,0,2.0,3.0,0.0,363.0,529.0


In [8]:
new_df=airbnb_df

In [9]:
##log transfromation
new_df.availability_365 += 0.000000001
new_df['availability_365'] = np.log10(new_df['availability_365'])

new_df.number_of_reviews += 0.000000001
new_df['number_of_reviews'] = np.log10(new_df['number_of_reviews'])

C:\Users\liang\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\liang\AppData\Local\Temp/ipykernel_7448/2942442649.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\liang\AppData\Local\Temp/ipykernel_7448/2942442649.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [10]:
indexidtracker = new_df[['id']]
latitudetracker = new_df[['latitude']]
longitudetracker = new_df[['longitude']]
##model 1 all variable
Model1  = new_df[['new_host_acceptance_rate','host_is_superhost','room_type','bedrooms'\
                  ,'beds','number_of_reviews','availability_365','neighbourhood_cleansed','neighbourhood_group_cleansed']]
##Model 2 room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat
Model2  = new_df[['room_type','bedrooms'\
                  ,'number_of_reviews','neighbourhood_cleansed','neighbourhood_group_cleansed']]
##model 3 room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat
Model3  = new_df[['room_type','number_of_reviews','neighbourhood_group_cleansed']]
y = new_df['price'].values
y = np.log10(y)


In [11]:
# Import train_test_split from sklearn
from sklearn.model_selection import train_test_split
m1_train, m1_test,m2_train,m2_test,m3_train,m3_test,y_train,y_test,id_train,id_test,lat_train,lat_test,long_train,long_test = train_test_split(Model1,Model2,Model3,y,indexidtracker,latitudetracker,longitudetracker,test_size = 0.2)

In [12]:
# Check the sample sizes for salesprice
print("Train Set :", m1_train.shape, y_train.shape)
print("Test Set  :", m1_test.shape, y_test.shape)
# Check the sample sizes overallqual
print("Train Set :", m2_train.shape, y_train.shape)
print("Test Set  :", m2_test.shape, y_test.shape)
# Check the sample sizes year built
print("Train Set :", m3_train.shape, y_train.shape)
print("Test Set  :", m3_test.shape, y_test.shape)

Train Set : (2936, 9) (2936,)
Test Set  : (734, 9) (734,)
Train Set : (2936, 5) (2936,)
Test Set  : (734, 5) (734,)
Train Set : (2936, 3) (2936,)
Test Set  : (734, 3) (734,)


In [13]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(m1_train, y_train)
# Predicting the Test set results
y_pred = lr.predict(m1_test)

In [14]:
model1df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_pred, 0)})
model1df.head(10)

,Actual,Predicted
0,48.0,122.0
1,541.0,205.0
2,281.0,187.0
3,228.0,160.0
4,217.0,130.0
5,96.0,104.0
6,190.0,158.0
7,143.0,161.0
8,33.0,70.0
9,120.0,191.0


In [15]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score

print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(m1_test))), 2))
m1rmse = np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(m1_test))), 2)
print('R2 score train:', np.round(r2_score(y_train, lr.predict(m1_train), multioutput='variance_weighted'), 2))
m1r2train= np.round(r2_score(y_train, lr.predict(m1_train), multioutput='variance_weighted'), 2)
print('R2 score test:', np.round(r2_score(y_test, lr.predict(m1_test), multioutput='variance_weighted'), 2))
m1r2test= np.round(r2_score(y_test, lr.predict(m1_test), multioutput='variance_weighted'), 2)

Price mean: 2.13
Price std: 0.37
RMSE: 0.28
R2 score train: 0.35
R2 score test: 0.37


## Model 2 room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat

In [16]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(m2_train, y_train)
# Predicting the Test set results
y_pred2 = lr.predict(m2_test)

In [17]:
model2df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_pred2, 0)})
model2df.head(10)

,Actual,Predicted
0,48.0,117.0
1,541.0,213.0
2,281.0,192.0
3,228.0,153.0
4,217.0,129.0
5,96.0,98.0
6,190.0,162.0
7,143.0,164.0
8,33.0,67.0
9,120.0,223.0


In [18]:
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(m2_test))), 2))
m2rmse = np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(m2_test))), 2)
print('R2 score train:', np.round(r2_score(y_train, lr.predict(m2_train), multioutput='variance_weighted'), 2))
m2r2train= np.round(r2_score(y_train, lr.predict(m2_train), multioutput='variance_weighted'), 2)
print('R2 score test:', np.round(r2_score(y_test, lr.predict(m2_test), multioutput='variance_weighted'), 2))
m2r2test= np.round(r2_score(y_test, lr.predict(m2_test), multioutput='variance_weighted'), 2)


Price mean: 2.13
Price std: 0.37
RMSE: 0.28
R2 score train: 0.34
R2 score test: 0.37


## Model 3 room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat

In [19]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(m3_train, y_train)
# Predicting the Test set results
y_pred3 = lr.predict(m3_test)

In [20]:
model3df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_pred3, 0)})
model3df.head(10)

,Actual,Predicted
0,48.0,118.0
1,541.0,257.0
2,281.0,181.0
3,228.0,188.0
4,217.0,118.0
5,96.0,118.0
6,190.0,180.0
7,143.0,175.0
8,33.0,57.0
9,120.0,257.0


In [21]:
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(m3_test))), 2))
m3rmse = np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(m3_test))), 2)
print('R2 score train:', np.round(r2_score(y_train, lr.predict(m3_train), multioutput='variance_weighted'), 2))
m3r2train =  np.round(r2_score(y_train, lr.predict(m3_train), multioutput='variance_weighted'), 2)
print('R2 score test:', np.round(r2_score(y_test, lr.predict(m3_test), multioutput='variance_weighted'), 2))
m3r2test= np.round(r2_score(y_test, lr.predict(m3_test), multioutput='variance_weighted'), 2)

Price mean: 2.13
Price std: 0.37
RMSE: 0.29
R2 score train: 0.3
R2 score test: 0.32


## Using Random forest to optimize

## Model 1 all variable

In [22]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=8, n_estimators = 100, random_state = 0)
rfr.fit(m1_train, y_train)
# Predicting the Test set results
y_predr1 = rfr.predict(m1_test)
df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_predr1, 0)})
df.head(10)

,Actual,Predicted
0,48.0,111.0
1,541.0,326.0
2,281.0,190.0
3,228.0,182.0
4,217.0,85.0
5,96.0,114.0
6,190.0,164.0
7,143.0,151.0
8,33.0,74.0
9,120.0,129.0


In [23]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, rfr.predict(m1_test))), 2))
m1rmserr =  np.round(np.sqrt(metrics.mean_squared_error(y_test, rfr.predict(m1_test))), 2)
print('R2 score train:', np.round(r2_score(y_train, rfr.predict(m1_train), multioutput='variance_weighted'), 2))
m1r2trainrr=  np.round(r2_score(y_train, rfr.predict(m1_train), multioutput='variance_weighted'), 2)
print('R2 score test:', np.round(r2_score(y_test, rfr.predict(m1_test), multioutput='variance_weighted'), 2))
m1r2testrr = np.round(r2_score(y_test, rfr.predict(m1_test), multioutput='variance_weighted'), 2)

Price mean: 2.13
Price std: 0.37
RMSE: 0.23
R2 score train: 0.7
R2 score test: 0.57


## Model 2 room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat

In [24]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=8, n_estimators = 100, random_state = 0)
rfr.fit(m2_train, y_train)
# Predicting the Test set results
y_predr2 = rfr.predict(m2_test)
df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_predr2, 0)})
df.head(10)

,Actual,Predicted
0,48.0,115.0
1,541.0,179.0
2,281.0,168.0
3,228.0,192.0
4,217.0,109.0
5,96.0,78.0
6,190.0,167.0
7,143.0,128.0
8,33.0,50.0
9,120.0,180.0


In [25]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, rfr.predict(m2_test))), 2))
m2rsmerr= np.round(np.sqrt(metrics.mean_squared_error(y_test, rfr.predict(m2_test))), 2)
print('R2 score train:', np.round(r2_score(y_train, rfr.predict(m2_train), multioutput='variance_weighted'), 2))
m2r2trainrr= np.round(r2_score(y_train, rfr.predict(m2_train), multioutput='variance_weighted'), 2)
print('R2 score test:', np.round(r2_score(y_test, rfr.predict(m2_test), multioutput='variance_weighted'), 2))
m2r2testrr=np.round(r2_score(y_test, rfr.predict(m2_test), multioutput='variance_weighted'), 2)

Price mean: 2.13
Price std: 0.37
RMSE: 0.25
R2 score train: 0.61
R2 score test: 0.51


## Model 3 room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat

In [26]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=8, n_estimators = 100, random_state = 0)
rfr.fit(m3_train, y_train)
# Predicting the Test set results
y_predr3 = rfr.predict(m3_test)
df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_predr3, 0)})
df.head(10)

,Actual,Predicted
0,48.0,117.0
1,541.0,255.0
2,281.0,166.0
3,228.0,203.0
4,217.0,117.0
5,96.0,117.0
6,190.0,146.0
7,143.0,128.0
8,33.0,43.0
9,120.0,255.0


In [27]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, rfr.predict(m3_test))), 2))
m3rsmerr=np.round(np.sqrt(metrics.mean_squared_error(y_test, rfr.predict(m3_test))), 2)
print('R2 score train:', np.round(r2_score(y_train, rfr.predict(m3_train), multioutput='variance_weighted'), 2))
m3r2trainrr = np.round(r2_score(y_train, rfr.predict(m3_train), multioutput='variance_weighted'), 2)
print('R2 score test:', np.round(r2_score(y_test, rfr.predict(m3_test), multioutput='variance_weighted'), 2))
m3r2testrr =  np.round(r2_score(y_test, rfr.predict(m3_test), multioutput='variance_weighted'), 2)

Price mean: 2.13
Price std: 0.37
RMSE: 0.29
R2 score train: 0.36
R2 score test: 0.33


## Selecting the Best Model For Data Driven Insight and Demonstration of concept using flask

In [28]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score
#Comparing Models
print("Model 1 All variable")
print("********************************************************************************************************************")
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:',m1rmse)
print('R2 score train:',m1r2train)
print('R2 score test:', m1r2test)
#RandomForest model 1 (All variable)
print("Model 1 All RR")
print("********************************************************************************************************************")
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:',m1rmserr)
print('R2 score train:',m1r2trainrr)
print('R2 score test:', m1r2testrr)
#Multivariate regression model 2(room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat)
print("Model 2 room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat")
print("********************************************************************************************************************")
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:',m2rmse)
print('R2 score train:',m2r2train)
print('R2 score test:', m2r2test)
#RandomForest model 2 (room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat)
print("Model 2 RR room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat")
print("********************************************************************************************************************")
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:',m2rsmerr)
print('R2 score train:',m2r2trainrr)
print('R2 score test:', m2r2testrr)
#Multivariate regression model 3 (room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat)
print("Model 3 room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat")
print("********************************************************************************************************************")
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:',m3rmse)
print('R2 score train:',m3r2train)
print('R2 score test:', m3r2test)
#randomforest model 3  (room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat)
print("Model 3 RR room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat")
print("********************************************************************************************************************")
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:',m3rsmerr)
print('R2 score train:',m3r2trainrr)
print('R2 score test:', m3r2testrr)


Model 1 All variable
********************************************************************************************************************
Price mean: 2.13
Price std: 0.37
RMSE: 0.28
R2 score train: 0.35
R2 score test: 0.37
Model 1 All RR
********************************************************************************************************************
Price mean: 2.13
Price std: 0.37
RMSE: 0.23
R2 score train: 0.7
R2 score test: 0.57
Model 2 room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat
********************************************************************************************************************
Price mean: 2.13
Price std: 0.37
RMSE: 0.28
R2 score train: 0.34
R2 score test: 0.37
Model 2 RR room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat
********************************************************************************************************************
Price mean: 2.13
Pr

__Conclusion__
1) Price prediction are performing well after using random forest model compared to multivariate regresion. Best score is 0.57(RR) vs 0.37(MVR)
2) Using categorical encoded data did improve the model.
3) Using more variable increase the accuracy of the model showing that, all the variable plays an important role in predicting price for property as there are a lot of factors affecting property pricing.
4) Scaling value and logging values improve model performance because , omitting scaling will basically result in models that are disproportionally influenced by the subset of features on a large scale.
5) When there are large number of features with less data-sets(with low noise), linear regressions may outperform Decision trees/random forests. But for this project random forest perform better because decision tree have better average accuracy. The model also used alot of categorical variable thus random forest perfom bettr.

__Random Forest Model 1 is the best model because the highest r2 and lowest rmse.__ Used by Airbnb
RMSE: 0.23
R2 score train: 0.7
R2 score test: 0.57
R2 score of 0.57 suggest that 57% represents a model that explains all the variation in the response variable around its mean. 
A low RMSE value of 0.23 indicates that the simulated and observed data are close to each other showing a better accuracy

__Random Forest Model 2 is the second best model__ Used for developing website for Airbnb host

__Deployment is one of the last stages of any machine learning project. This model have two different end-users, Airbnb host and Airbnb company
1) For Airbnb host, the team decided to target new Airbnb host. This model aims to help the new Airbnb host to guage/price their property at reasonable market rate. The model will be deployed using flask. A website will be provided for the user to input their proprety features and the website will output the predicted price. The team will deploy the second best model to this application because the team feel that that the best model should be used by Airbnb company as this model will give them competitive advantage over their competitors in the industry.

2) For Airbnb company, the model aims to help the company forecast revenue and guage whether the current price stragety is reasonable assuming the model is correct.

In [29]:
##Saving model into pickle for Airbnb host flask application

In [30]:
##Model to be used for flask
import pickle
rfr = RandomForestRegressor(max_depth=8, n_estimators = 100, random_state = 0)
rfr.fit(m2_train, y_train)
#make pickle file for our model
pickle.dump(rfr,open("model.pkl", 'wb'))

## generating predicted dataframe using the best model

In [31]:
rfr = RandomForestRegressor(max_depth=8, n_estimators = 100, random_state = 0)
rfr.fit(m1_train, y_train)
y_predr1 = rfr.predict(m1_test)
df = pd.DataFrame({'Actual Price': np.round(10 ** y_test, 0), 
                  'Predicted Price': np.round(10 ** y_predr1, 0)})


In [32]:
#to be concat later
df

,Actual Price,Predicted Price
0,48.0,111.0
1,541.0,326.0
2,281.0,190.0
3,228.0,182.0
4,217.0,85.0
...,...,...
729,227.0,181.0
730,83.0,95.0
731,220.0,183.0
732,144.0,131.0


In [33]:
m2_test

,room_type,bedrooms,number_of_reviews,neighbourhood_cleansed,neighbourhood_group_cleansed
3668,2,1.0,-9.000000e+00,19,0
898,0,1.0,-9.000000e+00,10,0
946,0,1.0,1.113943e+00,34,0
2475,0,1.0,4.342945e-10,4,0
3332,2,1.0,-9.000000e+00,30,0
...,...,...,...,...,...
2336,0,1.0,-9.000000e+00,4,0
3031,0,0.0,4.771213e-01,24,0
2165,0,1.0,-9.000000e+00,10,0
1191,0,1.0,8.450980e-01,10,0


In [34]:
##logging back all the different variable
exportm1test = m1_test
exportm1test.number_of_reviews = np.round(10 ** exportm1test.number_of_reviews, 0)
exportm1test.availability_365 = np.round(10 ** exportm1test.availability_365, 0)

C:\Users\liang\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
id_test

,id
3668,53910005.0
898,19535251.0
946,20234954.0
2475,41077161.0
3332,50986261.0
...,...
2336,40313848.0
3031,46446227.0
2165,38364769.0
1191,23506068.0


## inverting dictionary back to use dictionary to convert numerical to categorical

In [36]:
map1 = mapping_dict['neighbourhood_cleansed']
inv_map1 = {v: k for k, v in map1.items()}
exportm1test['neighbourhood_cleansed'].replace(inv_map1, inplace=True)

C:\Users\liang\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
map2 = mapping_dict['neighbourhood_group_cleansed']
inv_map2 = {v: k for k, v in map2.items()}
exportm1test['neighbourhood_group_cleansed'].replace(inv_map2, inplace=True)

In [38]:
map3 = mapping_dict['room_type']
inv_map3 = {v: k for k, v in map3.items()}
exportm1test['room_type'].replace(inv_map3, inplace=True)

In [39]:
map4 = mapping_dict['host_is_superhost']
inv_map4 = {v: k for k, v in map4.items()}
exportm1test['host_is_superhost'].replace(inv_map4, inplace=True)

In [40]:
exportm1test['id']=id_test['id']

C:\Users\liang\AppData\Local\Temp/ipykernel_7448/3601045518.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
df.reset_index(drop=True, inplace=True)
exportm1test.reset_index(drop=True, inplace=True)

In [42]:
finaldataframe = (pd.concat([df, exportm1test], axis=1))
tempdataframe = (pd.concat([lat_test,long_test],axis=1))

finaldataframe.reset_index(drop=True, inplace=True)
tempdataframe.reset_index(drop=True, inplace=True)

In [43]:
bigframe = (pd.concat([finaldataframe,tempdataframe], axis=1))

In [44]:
bigframe

,Actual Price,Predicted Price,new_host_acceptance_rate,host_is_superhost,room_type,bedrooms,beds,number_of_reviews,availability_365,neighbourhood_cleansed,neighbourhood_group_cleansed,id,latitude,longitude
0,48.0,111.0,0.0,f,Private room,1.0,1.0,0.0,365.0,Marine Parade,Central Region,53910005.0,1.30276,103.90158
1,541.0,326.0,100.0,t,Entire home/apt,1.0,1.0,0.0,266.0,Downtown Core,Central Region,19535251.0,1.28143,103.85232
2,281.0,190.0,100.0,t,Entire home/apt,1.0,1.0,13.0,19.0,Singapore River,Central Region,20234954.0,1.28869,103.84426
3,228.0,182.0,60.0,f,Entire home/apt,1.0,1.0,1.0,347.0,Bukit Merah,Central Region,41077161.0,1.27209,103.83428
4,217.0,85.0,96.0,f,Private room,1.0,1.0,0.0,338.0,Rochor,Central Region,50986261.0,1.30609,103.85188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,227.0,181.0,30.0,f,Entire home/apt,1.0,0.0,0.0,364.0,Bukit Merah,Central Region,40313848.0,1.27228,103.83394
730,83.0,95.0,84.0,f,Entire home/apt,0.0,1.0,3.0,45.0,Outram,Central Region,46446227.0,1.28349,103.84562
731,220.0,183.0,31.0,f,Entire home/apt,1.0,2.0,0.0,35.0,Downtown Core,Central Region,38364769.0,1.29672,103.85431
732,144.0,131.0,89.0,t,Entire home/apt,1.0,1.0,7.0,0.0,Downtown Core,Central Region,23506068.0,1.27633,103.84515


In [45]:
bigframe.to_csv('model1predicted.csv')